In [264]:
import pandas as pd
import json
from pandas.io.json import json_normalize
import seaborn as sb
import calendar
import datetime as dt

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

from pylab import rcParams
rcParams['figure.figsize'] = 35, 40

import fnmatch
import os
from tqdm import tqdm

from sklearn.preprocessing import scale

## Читаем данные

In [265]:
data=pd.read_pickle('MG_Sales.pickle',compression='gzip')

In [266]:
data.drop(['Контрагент'], axis=1, inplace=True)

## Визуализируем данные

In [267]:
def visualize_data(data,title,col,val,lim=-1):
    if lim>0:
        title+=' '+str(lim)+' максимальных позиций'

    data_table=data.pivot_table(index=['Год','Месяц','День'], columns=col, values=val, aggfunc=sum,fill_value=0,dropna=False)

    cols=data.groupby(col)[val].sum()
    cols.sort_values(ascending=False,inplace=True)
    
    
    maxval=0
    
    def convertData(ymd):
        y=ymd.name[0]
        m=ymd.name[1]
        d=ymd.name[2]
        try: 
            if (m==2)&(d==29): 
                return dt.datetime(1980,1,1)
            else:
                return dt.datetime(y,m,d) 
        except ValueError: 
            return dt.datetime(1980,1,1)

    #преобразуем дату
    data_table['Дата']=data_table.apply(convertData, axis=1)

    #добавляем колонки с датами
    jobDay=data[data['РабочиеВыходные']>2].groupby(['Год','Месяц','День'])['РабочиеВыходные'].max()
    celebrate=data[data['Праздник']==1].groupby(['Год','Месяц','День'])['Праздник'].max()

    #соединяем все
    data_table=pd.concat([data_table, jobDay,celebrate], axis=1).reset_index()#, join='inner'

    #удаляем лишнее
    data_table.drop(['Год','Месяц','День'], axis=1, inplace=True)
    data_table.drop(data_table[data_table['Дата']==dt.datetime(1980,1,1)].index, inplace=True)
    data_table.drop(data_table[data_table['Дата']>=dt.datetime.now()].index, inplace=True)
    
    df=data_table.set_index('Дата')
        
    init_notebook_mode(connected = True)
    _data = []    
    for column in cols.index:
        if lim==0: break
        lim-=1            
        
        _max=df[column].max()
        if _max>maxval: maxval=_max
        
        
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',            
            name = column,
            line=dict(
                shape='spline'
            )
        )                        
        _data.append(trace)
        
    for column,clr in zip(['РабочиеВыходные','Праздник'],['rgba(200, 200, 16, 0.5)','rgba(250, 100, 16, 0.5)']):
        trace = go.Bar(
            x = df.index,
            y = df[column].map(lambda val: maxval if val>0 else 0),
            name = column,
            opacity=0.6,
            marker=dict(
                color=clr,
                    line=dict(
                        color=clr,
                        width=0,
                    )
                )   
            )
        _data.append(trace)
    
    layout = dict(title = title)
    
    fig = dict(data = _data, layout = layout)
    iplot(fig, show_link=False)

In [268]:
#visualize_data(data,'Продажи','ЦветМеталла','Количество',5)


## Подготовка выборки

In [269]:
#веса на выходную переменную

cur_year=dt.datetime.now().year
data['yw']=data['Год'].map(lambda cell: 1.-(cur_year-cell)/40)

In [270]:
#перекодировка категориальных данных
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#получаем колонки категориальных характеристик
categorical_columns = [c for c in data.columns if data[c].dtype.name == 'object']
for col in categorical_columns:
    data[col]=le.fit_transform(data[col])

#делим выборку на обучающую и валидационную
#делитель выборки
startCurYear=data[data['Год']==cur_year].iloc[0].name

y=data['Количество']#целевая переменная
yw=data['yw']#веса целевой переменной

#шкалирование без целевой переменной с ее весами
columns=list(set(data.columns)-set(['Количество','yw']))
data=pd.DataFrame(data=scale(data.drop(['Количество','yw'],axis=1)),columns=columns)

In [271]:
#делим выборку
data_train_x=data[:startCurYear]
data_test_x=data[startCurYear:]

data_train_y=y[:startCurYear]
data_test_y=y[startCurYear:]

data_train_yw=yw[:startCurYear]
data_test_yw=yw[startCurYear:]

## Случайный лес

In [272]:
#from  sklearn.ensemble import RandomForestRegressor
#rf=RandomForestRegressor(n_estimators=200, n_jobs=-1,max_depth=5)
#rf.fit(data_train_x,data_train_y,data_train_yw.values)

## Регрессия стохастического градиентного спуска

In [273]:
from sklearn.linear_model import SGDRegressor
rf=SGDRegressor(n_iter=500)
rf.fit(data_train_x,data_train_y,None,None,data_train_yw.values)

SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=500, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

## Валидация


In [274]:
from sklearn.metrics import mean_squared_error
mean_squared_error(data_test_y, rf.predict(data_test_x))

0.012750849458613324

## Важные характеристики

In [275]:
#N=15
#print(str(N)+' важных характеристик')
#featureImportances=pd.DataFrame(data=rf.feature_importances_,index=data_train_x.columns)
#featureImportances=featureImportances.apply(lambda x: abs(x))
#featureImportances.sort_values([0],ascending=False,inplace=True)
#featureImportances[:N]